# An example showing how to generate bootstrapped error bars.

In [ ]:
import os
import sys
import time
import json

import pygsti
from pygsti.modelpacks.legacy import std1Q_XYI

%pylab inline

In [ ]:
#Get a GST estimate (similar to Tutorial 0)

# 1) get the target Model
target_model = std1Q_XYI.target_model()

# 2) get the building blocks needed to specify which operation sequences are needed
prep_fiducials, meas_fiducials = std1Q_XYI.prepStrs, std1Q_XYI.effectStrs
germs = std1Q_XYI.germs
maxLengths = [1,2,4,8,16]

# 3) generate "fake" data from a depolarized version of target_model
mdl_datagen = target_model.depolarize(op_noise=0.1, spam_noise=0.001)
listOfExperiments = pygsti.circuits.create_lsgst_circuits(
    target_model, prep_fiducials, meas_fiducials, germs, maxLengths)
ds = pygsti.data.simulate_data(mdl_datagen, listOfExperiments, num_samples=1000,
                                            sample_error="binomial", seed=1234)


results = pygsti.run_stdpractice_gst(ds, target_model, prep_fiducials, meas_fiducials,
                                    germs, maxLengths, modes="TP")
estimated_model = results.estimates['TP'].models['stdgaugeopt']

In [ ]:
print(target_model)

In [ ]:
estimated_model.operations

## Parametric Bootstrapping
Here we do parametric bootstrapping, as indicated by the 'parametric' argument below.
The output is eventually stored in the "mean" and "std" Models, which hold the mean and standard deviation values of the set of bootstrapped models (after gauge optimization).  It is this latter "standard deviation Model"
which holds the collection of error bars.  Note: due to print setting issues, the outputs that are printed here will not necessarily reflect the true accuracy of the estimates made.


In [ ]:
#The number of simulated datasets & models made for bootstrapping purposes.  
# For good statistics, should probably be greater than 10.
numGatesets=10

param_boot_models = pygsti.drivers.create_bootstrap_models(
                        numGatesets, ds, 'parametric', prep_fiducials, meas_fiducials, germs, maxLengths,
                        input_model=estimated_model, target_model=target_model,
                        start_seed=0, return_data=False,
                        verbosity=2)

In [ ]:
gauge_opt_pboot_models = pygsti.drivers.gauge_optimize_models(param_boot_models, estimated_model,
                                                                 plot=False) #plotting support removed w/matplotlib

In [ ]:
pboot_mean = pygsti.drivers.bootstrap._to_mean_model(gauge_opt_pboot_models, estimated_model)
pboot_std  = pygsti.drivers.bootstrap._to_std_model(gauge_opt_pboot_models, estimated_model)

#Summary of the error bars
print("Parametric bootstrapped error bars, with", numGatesets, "resamples\n")
print("Error in rho vec:") 
print(pboot_std['rho0'].to_vector(), end='\n\n')
print("Error in effect vecs:")
print(pboot_std['Mdefault'].to_vector(), end='\n\n')
print("Error in Gi:")
print(pboot_std['Gi'].to_vector(), end='\n\n')
print("Error in Gx:")
print(pboot_std['Gx'].to_vector(), end='\n\n')
print("Error in Gy:")
print(pboot_std['Gy'].to_vector())

## Non-parametric Bootstrapping
Here we do non-parametric bootstrapping, as indicated by the 'nonparametric' argument below.
The output is again eventually stored in the "mean" and "std" Models, which hold the mean and standard deviation values of the set of bootstrapped models (after gauge optimization).  It is this latter "standard deviation Model"
which holds the collection of error bars.  Note: due to print setting issues, the outputs that are printed here will not necessarily reflect the true accuracy of the estimates made.

(Technical note: ddof = 1 is by default used when computing the standard deviation -- see numpy.std -- meaning that we are computing a standard deviation of the sample, not of the population.)

In [ ]:
#The number of simulated datasets & models made for bootstrapping purposes.  
# For good statistics, should probably be greater than 10.
numModels=10

nonparam_boot_models = pygsti.drivers.create_bootstrap_models(
                          numModels, ds, 'nonparametric', prep_fiducials, meas_fiducials, germs, maxLengths,
                          target_model=target_model, start_seed=0, return_data=False, verbosity=2)

In [ ]:
gauge_opt_npboot_models = pygsti.drivers.gauge_optimize_models(nonparam_boot_models, estimated_model,
                                                                 plot=False) #plotting removed w/matplotlib

In [ ]:
npboot_mean = pygsti.drivers.bootstrap._to_mean_model(gauge_opt_npboot_models, estimated_model)
npboot_std  = pygsti.drivers.bootstrap._to_std_model(gauge_opt_npboot_models, estimated_model)

#Summary of the error bars
print("Non-parametric bootstrapped error bars, with", numGatesets, "resamples\n")
print("Error in rho vec:")
print(npboot_std['rho0'].to_vector(), end='\n\n')
print("Error in effect vecs:")
print(npboot_std['Mdefault'].to_vector(), end='\n\n')
print("Error in Gi:")
print(npboot_std['Gi'].to_vector(), end='\n\n')
print("Error in Gx:")
print(npboot_std['Gx'].to_vector(), end='\n\n')
print("Error in Gy:")
print(npboot_std['Gy'].to_vector())

In [ ]:
loglog(npboot_std.to_vector(),pboot_std.to_vector(),'.')
loglog(np.logspace(-4,-2,10),np.logspace(-4,-2,10),'--')
xlabel('Non-parametric')
ylabel('Parametric')
xlim((1e-4,1e-2)); ylim((1e-4,1e-2))
title('Scatter plot comparing param vs. non-param bootstrapping error bars.')